In [1]:
import os, sys

sys.path.insert(0, os.path.abspath('..'))
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload 2

In [2]:
from google.cloud import storage
from google.oauth2 import service_account
from labelbox import Client as LabelboxClient

from utils import gcp_utils
from utils.data_management import get_annotation_path
from utils.labelbox_utils import get_annotation_objects_from_data_row_export, get_geojson_fc_from_annotation_objects

import geopandas as gpd
import json
import uuid

2023-11-01 18:51:40:rastervision.pipeline.rv_config: WARNING - Root temporary directory cannot be used: /opt/data/tmp. Using root: /tmp/tmprcp00yv9


In [19]:
import labelbox.types as lb_types
from labelbox import MALPredictionImport
from labelbox import Dataset

In [ ]:
from project_config import GCP_PROJECT_NAME, BUCKET_NAME
LABELBOX_API_KEY = os.getenv('LABELBOX_API_KEY')

gcp_client = storage.Client(project=GCP_PROJECT_NAME)
labelbox_client = LabelboxClient(api_key=LABELBOX_API_KEY)

In [54]:
def get_list_of_annotations(annotated_objects, name = 'sandmine'):
    annotations = []
    for obj in annotated_objects:
        points = [lb_types.Point(x = f[0], y = f[1]) for f in obj['geojson']['coordinates'][0]]

        polygon_annotation = lb_types.ObjectAnnotation(
            name="sandmine",  # must match your ontology feature's name 
            value=lb_types.Polygon(  # Coordinates for the vertices of your polygon
                points = points))
        annotations.append(polygon_annotation)
        
    return annotations

#### Get Exports

In [6]:
#### Id of existing project
LABELBOX_PROJECT_ID = "cllbeyixh0bxt07uxfvg977h3"

project = labelbox_client.get_project(LABELBOX_PROJECT_ID)
export_params = {"project_details": True}
export = project.export_v2(params=export_params)

export.wait_till_done()
if export.errors:
    print("Errors:")
    print(export.errors)
else:
    export_full_project = export.result
    print(f"Finished without errors. Total export contains {len(export_full_project)} data rows.")
    print(len(export_full_project))


Finished without errors. Total export contains 186 data rows.
186


#### Batch and Load All The Images

In [25]:
new_project_id = "clogjpfrf0j9f070ja8fmczr3"

In [28]:
#### Get all "DONE" rows
global_keys = []

for data_row_export in export_full_project:
    global_key_data_row = data_row_export['data_row']['global_key']
    j+=1
    workflow_status = data_row_export['projects'][LABELBOX_PROJECT_ID]['project_details']['workflow_status']
    # We filter depending on workflow_status
    if workflow_status in ["IN_REWORK", "TO_LABEL", "IN_REVIEW"]:
        continue
    
    assert workflow_status == "DONE"
    
    global_keys.append(global_key_data_row)

In [30]:
new_project = labelbox_client.get_project(new_project_id)
batch = new_project.create_batch(
  "Batch1", # Each batch in a project must have a unique name
  global_keys= global_keys, # Paginated collection of data row objects, list of data row ids or global keys
  priority=5 # priority between 1(Highest) - 5(lowest)
)

print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-11-02 02:10:11+00:00",
    "name": "Batch1",
    "size": 80,
    "uid": "f43ac5a0-7924-11ee-b66a-57ff7be1430b",
    "updated_at": "2023-11-02 02:10:11+00:00"
}>


#### Prepare annotations for export

In [55]:
labels =[]
for data_row_export in export_full_project:
    global_key_data_row = data_row_export['data_row']['global_key']
    j+=1
    workflow_status = data_row_export['projects'][LABELBOX_PROJECT_ID]['project_details']['workflow_status']
    # We filter depending on workflow_status
    if workflow_status in ["IN_REWORK", "TO_LABEL", "IN_REVIEW"]:
        continue
    
    assert workflow_status == "DONE"
    
    print(f"Processing annotations for {global_key_data_row}")
    annotated_objects = get_annotation_objects_from_data_row_export(data_row_export)
    annotations = get_list_of_annotations(annotated_objects)
    
    img_url = eval(data_row_export['data_row']['row_data'])['tileLayerUrl']
    tile_layer_url = eval(data_row_export['data_row']['row_data'])['alternativeLayers'][0]['tileLayerUrl']
    
    geojson_name = "_".join(global_key_data_row.split("_")[:4])
    geojson_file = f"""https://storage.googleapis.com/sand_mining_median/labels/{geojson_name}_median/shp/{geojson_name}.geojson"""
    df_gpd = gpd.read_file(geojson_file)
    x1, y1, x2, y2 = df_gpd.bounds.values[0]

    top_left_bound = lb_types.Point(x=x1, y=y2)
    bottom_right_bound = lb_types.Point(x=x2, y=y1)

    epsg = lb_types.EPSG.EPSG4326
    bounds = lb_types.TiledBounds(epsg=epsg, bounds=[top_left_bound, bottom_right_bound])

    tile_layer = lb_types.TileLayer(
        url=tile_layer_url
    )
    

    labels.append(
        lb_types.Label(
    #         data=lb_types.TileLayer(global_key = global_key_data_row , url = tile_layer_url),
            data = lb_types.TiledImageData(
                global_key=global_key_data_row,
                tile_layer= tile_layer,
                tile_bounds=bounds,
                zoom_levels=[4, 20]
            ),
            annotations = annotations
        )
    )

Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-05-01_rgb_median
Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-02-01_rgb_median
Processing annotations for Ken_Banda_80-35_25-68_2022-06-01_rgb_median
Processing annotations for Ken_Banda_80-35_25-68_2022-01-01_rgb_median
Processing annotations for Narmada_Sehore_77-32_22-56_2022-01-01_rgb_median
Processing annotations for Narmada_Sehore_77-32_22-56_2022-10-01_rgb_median
Processing annotations for Tawa_Hoshangabad_77-80_22-74_2022-04-01_rgb_median
Processing annotations for Tawa_Hoshangabad_77-80_22-74_2022-01-01_rgb_median
Processing annotations for Sone_Rohtas_84-21_24-91_2022-02-01_rgb_median
Processing annotations for Sone_Rohtas_84-21_24-91_2022-05-01_rgb_median
Processing annotations for Sone_Rohtas_83-86_24-46_2023-01-01_rgb_median
Processing annotations for Sone_Rohtas_83-86_24-46_2023-06-01_rgb_median
Processing annotations for Bhargavi_Khordha_85-88_20-26_2018-12-01_rgb_median
Processing annotations

In [60]:
upload_job = MALPredictionImport.create_from_objects(
  client = labelbox_client, 
  project_id = new_project.uid, 
  name="mal_import_job"+str(uuid.uuid4()), 
  predictions=labels
)

upload_job.wait_until_done();
print("Errors:", upload_job.errors)
print("Status of uploads: ", upload_job.statuses)

Errors: []
Status of uploads:  [{'uuid': 'ec57d325-bacc-406d-9700-87d4edb39c8a', 'dataRow': {'id': 'cllbefzr706lu075o4tjp2vil', 'globalKey': 'Kathajodi_Cuttack_85-85_20-44_2022-05-01_rgb_median'}, 'status': 'SUCCESS'}, {'uuid': '97c05987-df52-461b-b53a-04e21e3903ef', 'dataRow': {'id': 'cllbefzr706lu075o4tjp2vil', 'globalKey': 'Kathajodi_Cuttack_85-85_20-44_2022-05-01_rgb_median'}, 'status': 'SUCCESS'}, {'uuid': '0e00b905-0844-45ff-96c8-9813a616a3ed', 'dataRow': {'id': 'cllbefzr706lu075o4tjp2vil', 'globalKey': 'Kathajodi_Cuttack_85-85_20-44_2022-05-01_rgb_median'}, 'status': 'SUCCESS'}, {'uuid': 'd6e5a2e6-e7db-450a-b6d1-b8fa3d4b5e85', 'dataRow': {'id': 'cllbefzr706lu075o4tjp2vil', 'globalKey': 'Kathajodi_Cuttack_85-85_20-44_2022-05-01_rgb_median'}, 'status': 'SUCCESS'}, {'uuid': '52a876ec-1c39-4583-bc88-d08b78309fdb', 'dataRow': {'id': 'cllbefzr706lu075o4tjp2vil', 'globalKey': 'Kathajodi_Cuttack_85-85_20-44_2022-05-01_rgb_median'}, 'status': 'SUCCESS'}, {'uuid': '36bd0caf-c514-4ec7-bc49